## Entropy in word level

Entropy is the measure of uncertainty or it's providnig the information about the whole system. The minimum entropy = 0 and maximum is log(k), where k is the number of levels or class have in the system. So, as an example if I have two different category then the minimum entropy is = 0 and in log_2(2) = 1.

The maximum value of entropy is logk, where k is the number of categories you are using. Its numeric value will naturally depend on the base of logarithms you are using.

Using base 2 logarithms as an example, as in the question: log21 is 0 and log22 is 1, so a result greater than 1 is definitely wrong if the number of categories is 1 or 2. A value greater than 1 will be wrong if it exceeds log2k.

In view of this it is fairly common to scale entropy by logk, so that results then do fall between 0 and 1,

# Functions

### Calculating the Entropy of the text 

We are calculating the entropy the define the predictibility fromt the text. The entropy more close to 1 it less predictable or more random. So, the prediction would be better for the text where the entropy is less.

In [36]:
# Finding the word level or charecter level entropy!!!
import math
from collections import Counter

def Entropy_of_text(content):
    # The data are the tokens of the sentences
    
    data = content.split(' ')

    base =len(data)

    if len(data) <= 1:
        return 0

    counts = Counter()

    for d in data:
        counts[d] += 1

    ent = 0

    probs = [float(c) / len(data) for c in counts.values()]
    for p in probs:
        if p > 0.:
            ent -= p * math.log(p, base)

    return ent

# Output will be always withing 0 to 1 (we are using the log base = the number of class to make the output in 0-1)

### Transition Probability Matrix

In [37]:
import scipy.sparse
from sklearn.preprocessing import normalize
import numpy as np
import re
import warnings


def create_ngrams(tokens,n):
    sequences = [tokens[i:] for i in range(n)]
    ngrams = [' '.join(ngram) for ngram in list(zip(*sequences))]
    return ngrams, list(set(ngrams))


def create_word_mapping(values_list):
        values_list.append('<| unknown |>')
        value2ind = {value: ind for ind, value in enumerate(values_list)}
        ind2value = dict(enumerate(values_list))
        return value2ind, ind2value

def create_transition_prob_matrix(input_text,n):
    
    # n is for selecting the ngram
    import re

    punctuation_pad = '!?.,:-;'
    punctuation_remove = '"()_\n'


    content_preprocess = re.sub(r'(\S)(\n)(\S)', r'\1 \2 \3', input_text)
    # Removing charecter like \n\n
    content_preprocess = content_preprocess.translate(str.maketrans('', '', punctuation_remove))
    # Considering , . as different charecter to be used as a state in the transition probability matri
    content_preprocess = content_preprocess.translate(str.maketrans({key: ' {0} '.format(key) for key in punctuation_pad}))
    # Removing extra space between words or charecters
    content_preprocess = re.sub(' +', ' ', content_preprocess)
    # remove leading and trailing whitespaces
    content = content_preprocess.strip()
    
    tokens = content.split(' ')
    token_dist = list(set(tokens))
    

    
    ngram, ngram_dist = create_ngrams(tokens,n)
    
    token2ind, ind2token = create_word_mapping(token_dist) # distinct token
    ngram2ind, ind2ngram = create_word_mapping(ngram_dist) # distinct ngram
    
    #tokens_ind = [token2ind[token] if token in token2ind.keys() else token2ind['<| unknown |>']
     #                      for token in tokens]

    
    
    row_ind, col_ind, values = [], [], []

    for i in range(len(tokens[:-n])):
        ngram = ' '.join(tokens[i:i + n])
        ngram_ind = ngram2ind[ngram]
        next_word_ind = token2ind[tokens[i + n]]

        row_ind.extend([ngram_ind])
        col_ind.extend([next_word_ind])
        values.extend([1])

    S = scipy.sparse.coo_matrix((values, (row_ind, col_ind)), shape=(len(ngram2ind), len(token2ind)))
    
    P = normalize(S, norm='l1', axis=1)
    return P



    
def powr(P,order):
    import numpy as np
    k = len(P)
    M = np.identity(k)
    for i in range(order):
        M = np.dot(M,P)
    return M


def transition_probability_matrix(input_text,n,order):
    if order == 1:
        P = create_transition_prob_matrix(input_text,n)
    else:
        P = create_transition_prob_matrix(input_text,n)
        P = P.toarray()
        P = powr(P,order)
        P = scipy.sparse.coo_matrix(P)
        P = P.tocsr()
    return P

### Generating next word

In [38]:
def pre_process(input_text):
    # n is for selecting the ngram
    import re

    punctuation_pad = '!?.,:-;'
    punctuation_remove = '"()_\n'


    content_preprocess = re.sub(r'(\S)(\n)(\S)', r'\1 \2 \3', input_text)
    # Removing charecter like \n\n
    content_preprocess = content_preprocess.translate(str.maketrans('', '', punctuation_remove))
    # Considering , . as different charecter to be used as a state in the transition probability matri
    content_preprocess = content_preprocess.translate(str.maketrans({key: ' {0} '.format(key) for key in punctuation_pad}))
    # Removing extra space between words or charecters
    content_preprocess = re.sub(' +', ' ', content_preprocess)
    # remove leading and trailing whitespaces
    content = content_preprocess.strip()
    return content
    


def get_ngram(input_text,n):

    content = pre_process(input_text)    
    tokens = content.split(' ')
    token_dist = list(set(tokens))
    
    sequences = [tokens[i:] for i in range(n)]
    ngrams = [' '.join(ngram) for ngram in list(zip(*sequences))]
    
    
    #ngram, ngram_dist = create_ngrams(tokens,n)
    
    return ngrams

def check_prefix(input_text,prefix,n):
    prefix_list = prefix.split(' ')[-n:]
    ngrams = get_ngram(input_text,n)
    if len(prefix_list) < n:
        warnings.warn(
                'Prefix is too short, please provide prefix of length: %d. Random ngram used instead.' % n)
        return np.random.choice(ngrams)
    else:
        prefix = ' '.join(prefix_list)
        if prefix in ngrams:
            return prefix
        else:
            warnings.warn(
                    'Prefix is not included in ngrams of the model. Provide another prefix. Random ngram used instead.')
            return np.random.choice(ngrams)

        
def return_next_word(input_text,prefix,n):
    prefix = check_prefix(input_text,prefix,n)
    content = pre_process(input_text)
 
    
    tokens = content.split(' ')
    token_dist = list(set(tokens))
    

    
    ngram, ngram_dist = create_ngrams(tokens,n)
    token2ind, ind2token = create_word_mapping(token_dist) # distinct token
    ngram2ind, ind2ngram = create_word_mapping(ngram_dist) # distinct ngram
    
    # Transition prob matrix
    transition_matrix_prob = create_transition_prob_matrix(input_text,n)
    #transition_matrix_prob = transition_probability_matrix(input_text,n,order)
    prefix_ind = ngram2ind[prefix]
    weights = transition_matrix_prob[prefix_ind].toarray()[0]
    token_ind = np.random.choice(range(len(weights)), p=weights)
    next_word = ind2token[token_ind]
    return next_word

def reverse_preprocess(text):
    text_reverse = re.sub(r'\s+([!?"\'().,;-])', r'\1', text)
    text_reverse = re.sub(' +', ' ', text_reverse)
    return text_reverse



def generate_sequence(input_text,prefix,n, k):
    prefix = check_prefix(input_text,prefix,n)
    sequence = prefix.split(' ')

    for i in range(k):
        next_word = return_next_word(input_text,prefix,n)
        sequence.append(next_word)
        prefix = ' '.join(sequence[-n:])

    return reverse_preprocess(' '.join(sequence))

# Application

## Game of throns

### Entropy

In [39]:
path = 'E:/NMT MS/Spring 22/MATH 486/Project/game_of_thrones/game_of_thrones.txt'
input_text = open(path, 'r', encoding='utf-8').read()
Entropy_of_text(content = input_text)

0.5713295213655606

### Next word

In [40]:
# n = number of ngram consider ... n= 1 means only individual word used, 2 means it will create bigram and then create the word
# k = number of word or character need to be generated.

generate_sequence(input_text,prefix = "my",n = 1,k=5)

"my mother says I'll kill,"

## The Philosopher's Stone

In [41]:
path = "E:/NMT MS/Spring 22/MATH 486/Project/Harry Potter/Book 1 - The Philosopher's Stone.txt"
input_text = open(path, 'r', encoding='utf-8').read()
Entropy_of_text(content = input_text)

0.6543526856202017

In [42]:
generate_sequence(input_text,prefix = "my",n = 1,k=5)

'my toad, boiled potatoes,'

## Macbeth

In [43]:
path = "E:/NMT MS/Spring 22/MATH 486/Project/Books_EngFr/Books_EngFr/English/shakespeare/Macbeth.txt"
input_text = open(path, 'r', encoding='utf-8').read()
Entropy_of_text(content = input_text)

0.7103438046543191

In [44]:
generate_sequence(input_text,prefix = "my",n = 1,k=5)

'my Prayers with horrorsDirenesse familiar to'